# Titanic Predictions

From the previous file, I found that KSVM worked the best, I will implement it here to give guesses for the unknown data.

First, I have to import the libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Import dataset

In [2]:
dataset = pd.read_csv("train.csv")

In [3]:
dataset=dataset.drop(["PassengerId","Name","Ticket","Cabin", "Parch", "SibSp"], axis=1)
dataset = dataset[dataset['Embarked'].notna()] #found some NaN in Embarked too

Ok, so Survived is our y. Our numerical features are Age, Fare and Sibsp, and categorical features are Pclass, Sex and Embarked 

In [4]:
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

In [5]:
#Imputer to fill in missing numerical data
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#the missing values are np.nan, and we replace them with the 'mean' of other elements in the column
imputer.fit(X[:, 2:4])
X[:, 2:4] = imputer.transform(X[:, 2:4])

In [6]:
X

array([[3, 'male', 22.0, 7.25, 'S'],
       [1, 'female', 38.0, 71.2833, 'C'],
       [3, 'female', 26.0, 7.925, 'S'],
       ...,
       [3, 'female', 28.0, 23.45, 'S'],
       [1, 'male', 26.0, 30.0, 'C'],
       [3, 'male', 32.0, 7.75, 'Q']], dtype=object)

We have to wait to apply standard scaling until after we split the data into training and test sets.

In [7]:
#Encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1,4])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[0.0 0.0 1.0 ... 1.0 22.0 7.25]
 [1.0 0.0 0.0 ... 0.0 38.0 71.2833]
 [0.0 0.0 1.0 ... 1.0 26.0 7.925]
 ...
 [0.0 0.0 1.0 ... 1.0 28.0 23.45]
 [1.0 0.0 0.0 ... 0.0 26.0 30.0]
 [0.0 0.0 1.0 ... 0.0 32.0 7.75]]


In [9]:
X.shape

(889, 10)

We don't split into training and test sets, X and y are our training sets.

In [10]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [11]:
X_train = X
y_train = y

We have to load the test set from the .csv file.

In [12]:
dataset2 = pd.read_csv("test.csv")
dataset2 = dataset2.drop(["PassengerId","Name","Ticket","Cabin", "Parch", "SibSp"], axis=1)

X_test = dataset2.values

In [13]:
dataset2.values

array([[3, 'male', 34.5, 7.8292, 'Q'],
       [3, 'female', 47.0, 7.0, 'S'],
       [2, 'male', 62.0, 9.6875, 'Q'],
       ...,
       [3, 'male', 38.5, 7.25, 'S'],
       [3, 'male', nan, 8.05, 'S'],
       [3, 'male', nan, 22.3583, 'C']], dtype=object)

In [14]:
X_test

array([[3, 'male', 34.5, 7.8292, 'Q'],
       [3, 'female', 47.0, 7.0, 'S'],
       [2, 'male', 62.0, 9.6875, 'Q'],
       ...,
       [3, 'male', 38.5, 7.25, 'S'],
       [3, 'male', nan, 8.05, 'S'],
       [3, 'male', nan, 22.3583, 'C']], dtype=object)

In [15]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#the missing values are np.nan, and we replace them with the 'mean' of other elements in the column
imputer.fit(X_test[:, 2:4])
X_test[:, 2:4] = imputer.transform(X_test[:,2:4])

In [16]:
X_test

array([[3, 'male', 34.5, 7.8292, 'Q'],
       [3, 'female', 47.0, 7.0, 'S'],
       [2, 'male', 62.0, 9.6875, 'Q'],
       ...,
       [3, 'male', 38.5, 7.25, 'S'],
       [3, 'male', 27.0, 8.05, 'S'],
       [3, 'male', 27.0, 22.3583, 'C']], dtype=object)

In [17]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1,4])], remainder='passthrough')
X_test = np.array(ct.fit_transform(X_test))

In [18]:
X_test

array([[0.0, 0.0, 1.0, ..., 0.0, 34.5, 7.8292],
       [0.0, 0.0, 1.0, ..., 1.0, 47.0, 7.0],
       [0.0, 1.0, 0.0, ..., 0.0, 62.0, 9.6875],
       ...,
       [0.0, 0.0, 1.0, ..., 1.0, 38.5, 7.25],
       [0.0, 0.0, 1.0, ..., 1.0, 27.0, 8.05],
       [0.0, 0.0, 1.0, ..., 0.0, 27.0, 22.3583]], dtype=object)

In [19]:
X_test.shape

(418, 10)

## Feature Scaling

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, -2:] = sc.fit_transform(X_train[:, -2:])
X_test[:, -2:] = sc.transform(X_test[:, -2:])

# KSVM Model

After analysis, it looks like KSVM with radial basis functions has the best performance on the test set so I will use that.

Can't do any estimate for the accuracy because we don't know the correct answers.

In [25]:
from sklearn.svm import SVC
ksvm = SVC(kernel = 'rbf', random_state = 0)
ksvm.fit(X_train, y_train)

SVC(random_state=0)

In [26]:
X_train[0,:]

array([0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.5636740664747024,
       -0.5002397510893564], dtype=object)

In [23]:
len(X_test[0,:])

10

In [37]:
y_out = ksvm.predict(X_test)
y_out

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Now, just need to export the prediction. What is it supposed to look like?

In [28]:
example = pd.read_csv("gender_submission.csv")
example.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


So, I need to do some stuff to get the correct format for the output...

In [30]:
X_test

array([[0.0, 0.0, 1.0, ..., 0.0, 0.39952204614424086,
        -0.4885786819404943],
       [0.0, 0.0, 1.0, ..., 1.0, 1.3627181587631843, -0.5052730164333057],
       [0.0, 1.0, 0.0, ..., 0.0, 2.5185534939059164, -0.4511654139858496],
       ...,
       [0.0, 0.0, 1.0, ..., 1.0, 0.7077448021823027, -0.5002397510893564],
       [0.0, 0.0, 1.0, ..., 1.0, -0.17839562142712512,
        -0.4841333019887182],
       [0.0, 0.0, 1.0, ..., 0.0, -0.17839562142712512,
        -0.19606341990539283]], dtype=object)

In [32]:
tempdata=pd.read_csv("test.csv")

In [45]:
y_df = pd.DataFrame(y_out,columns = ["Survived"])
y_df

,Survived
0,0
1,0
2,0
3,0
4,0
...,...
413,0
414,1
415,0
416,0


In [55]:
output = pd.DataFrame(tempdata['PassengerId'])
output = pd.concat([output,y_df],axis=1)
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


Ok, that looks right, now I just need to export that. I think all I have to do is...

In [56]:
output.to_csv("Submission.csv",index=False)

Can I see if this was saved correctly?

In [57]:
pd.read_csv("Submission.csv")

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


The score on Kaggle was 0.78229, which isn't really that great.